<a href="https://colab.research.google.com/github/cristobalroldan/TareaGIO/blob/main/Newton_n2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
import numpy as np 
import math
import time
from tabulate import tabulate
from prettytable import PrettyTable
from termcolor import colored
import numpy.linalg as npla

def f(x): 
    i = 1
    totalsum = np.float128(0.0)
    totalexp = np.float128(0.0)
    aux = np.float128(0.0)
    aux2 = np.float128(0.0)
    while i <= m:
        totalexp = 0
        j = 1
        while j <= n:
            aux = np.float128(((a[i-1][j-1])*x[j-1]) + b[i-1])
            totalexp = np.float128(totalexp) + np.float128(aux)
            j = j + 1
        aux2 = np.float128(np.exp(np.float128(totalexp)))
        #print(aux)
        totalsum = np.float128(totalsum + aux2)
        #print(totalsum)
        i = i + 1
    return np.log(np.float64(totalsum)) # retorna float 64 bits


def fsinLog(x): 
    i = 1
    totalsum = np.float128(0.0)
    totalexp = np.float128(0.0)
    aux = np.float128(0.0)
    aux2 = np.float128(0.0)
    while i <= m:
        totalexp = 0
        j = 1
        while j <= n:
            aux = np.float128(((a[i-1][j-1])*x[j-1]) + b[i-1])
            totalexp = np.float128(totalexp + aux)
            j = j + 1
        aux2 = np.exp(np.float128(totalexp))
        #print(aux)
        totalsum = np.float128(totalsum + aux2)
        #print(totalsum)
        i = i + 1
    return np.float128(totalsum)

def fdx(x):
    # print(x)
    i = 1
    totalsum = np.float128(0.0)
    totalexp = np.float128(0.0)
    aux = np.float128(0.0)
    aux2 = np.float128(0.0)
    while i <= m:
        totalexp = 0
        j =1
        while(j <= n):
          aux = np.float128(a[i-1][j-1]*x[j-1] + b[i-1])
          totalexp = np.float128(totalexp) + np.float128(aux)
          j = j+1
        aux2 = np.float128(np.exp(np.float128(totalexp))*a.item(i-1,0))
        #print(aux2)
        totalsum = np.float128(totalsum) + np.float128(aux2)
        # print(totalsum)
        i = i + 1
    
    i = 1
    totalsum2 = np.float128(0.0)
    totalexp = np.float128(0.0)
    aux = np.float128(0.0)
    aux2 = np.float128(0.0)
    while i <= m:
        totalexp = 0
        j =1
        while(j <= n):
          aux = np.float128(a[i-1][j-1]*x[j-1] + b[i-1])
          totalexp = np.float128(totalexp) + np.float128(aux)
          j = j +1
        aux2 = np.float128(np.exp(np.float128(totalexp))*a.item(i-1,1))
        #print(aux2)

        totalsum2 = np.float128(totalsum2) + np.float128(aux2)
        # print(totalsum)
        i = i + 1

    return np.array([(1/fsinLog(x))*totalsum, (1/fsinLog(x))*totalsum2], dtype=float) #retorna float64

def Hessian(x0,fdx): # (punto actual, primera derivada)
  dt=1e-8
  dxdx=( fdx( [x0[0]+dt, x0[1]] )[0] - fdx(x0)[0] ) / dt
  dxdy=( fdx( [ x0[0]+dt , x0[1] ] )[1] - fdx(x0)[1] ) / dt
  dydx=( fdx( [ x0[0] , x0[1]+dt ] )[0] - fdx(x0)[0] ) / dt
  dydy=( fdx( [ x0[0] , x0[1]+dt ] )[1] - fdx(x0)[1] ) / dt
  return[[dxdx,dxdy],[dydx,dydy]]

def backtrack5(x0, f, fdx, t = 1, alpha = 0.005, beta = 0.2):
    
    point = x0 #Necessary to ensure matrix form
    #print(point)
    #while f(point - np.dot(t,fdx(point))) > f(point) + alpha * t * np.asscalar(np.dot(np.transpose(fdx(point)), -1*fdx(point))):
    while f(point - np.dot(t,-fdx)) > f(point) + alpha * t * (np.dot(np.transpose(-fdx), 1*fdx)):
      t *= beta
      #print(f(point - np.dot(t,fdx)))
    return t


def lambda_sq(fdx, Hessian, point):
    lambda_sq1 = np.dot(fdx(point).T, np.dot(npla.pinv( Hessian(point,fdx) ) , fdx(point)  )) 
    return lambda_sq1.item() 
     
def delta_x(fdx, Hessian, point):
    delta_x =- np.dot(npla.pinv(Hessian(point,fdx)) , fdx(point))
    return np.array(delta_x)

def newtons_method(x, eps=0.0001, max_iters=25):
    start = time.time()
    iter = 1
    table = PrettyTable(['iter','x0', 'f(x0)', 'fdx(x0)'])
    lmb_sq = lambda_sq(fdx, Hessian, x)
    #print(x, f(x), fdx(x), 0, iter)
    while(( ((lmb_sq))**2/2.0) > eps):
        dlt_x = delta_x(fdx, Hessian, x) # paso
        lmb_sq = lambda_sq(fdx, Hessian, x)  #decremento

        t =backtrack5(x, f, dlt_x)  # t óptimo
        x = np.array((x + np.dot(t , dlt_x))) # nuevo x
        #print(x, f(x), fdx(x), t, iter)
        table.add_row([iter, x, f(x), dlt_x])
        iter = 1 + iter
        if(iter > max_iters):
            break
    print(table)
    print("Metodo completado en",iter-1 ,"iteraciones")
    end = time.time()
    return iter-1, end - start, f(x)        
    

# print(grad(x, 100))

print(colored("------------------------------- Instancias de Testeo -------------------------------", attrs=['bold']))

m = 100
n =2
limInf = -10
limSup = 10
print(colored("\n+ CONFIGURACION 1 -> m =" + str(m) +", n =" +str(n) + ", U[" + str(limInf) + "," + str(limSup) +"]", attrs=['bold']))
totalTime = 0
totalIter = 0
totalResult = 0
detailTime = np.zeros(10)
detailResult = np.zeros(10)
for i in range(10):
  print(colored("+ Instancia N:", attrs=['bold']), i + 1)
  m = 100
  n = 2
  a = np.random.uniform(-10,10, size=(m,n))
  np.around(a,2,a)
  b = np.random.uniform(-10,10, m)
  np.around(b,2,b)
  x=[1,1] # x fijo
  # print(x)
  Results = newtons_method(x)
  detailTime[i] = Results[1]
  detailResult[i] = Results[2]
  totalTime = totalTime + Results[1]
  totalIter = totalIter + Results[0]
  totalResult = totalResult + Results[2]
  print("\n")
  time.sleep(1)

print(colored("\n+ DETALLE RESULTADOS CONFIGURACION 1", attrs=['bold']))
table2 = PrettyTable(['Instancia','t[s]', 'f(x0)'])
for i in range(10):
  table2.add_row([i+1, round(detailTime[i],3),round(detailResult[i],3)])
print(table2)

print(colored("\n+ RESULTADOS CONFIGURACION 1", attrs=['bold']))
print("Numero de iteraciones TOTALES", totalIter)
print("Numero de iteraciones PROMEDIO", round(totalIter/10,2))
print("Tiempo TOTAL de ejecucion", round(totalTime,2), "segundos")
print("Tiempo PROMEDIO de ejecucion x Instancia", round(totalTime/10,2) , "segundos")
print("Optimo PROMEDIO ", round(totalResult/10,3) )


------------------------------- Instancias de Testeo -------------------------------

+ CONFIGURACION 1 -> m =100, n =2, U[-10,10]
+ Instancia N: 1
+------+---------------------------+--------------------+-----------------------------+
| iter |             x0            |       f(x0)        |           fdx(x0)           |
+------+---------------------------+--------------------+-----------------------------+
|  1   |  [1.184997   0.57410374]  | 27.70918002768517  | [ 23.12462535 -53.23703294] |
|  2   | [-1.46791168  0.12487173] | 27.293568852032227 |  [-2.65290868 -0.449232  ]  |
|  3   | [-0.8580198   0.15296093] | 21.988025276227823 |  [15.24729697  0.70222983]  |
|  4   |  [0.40450392 0.05928283]  | 21.869282995138697 |  [ 1.26252372 -0.0936781 ]  |
|  5   | [ 0.11414893 -0.0529871 ] | 20.502874898246322 |  [-7.25887464 -2.80674813]  |
|  6   | [-0.31071217 -0.22729545] |  19.5685964955023  |  [-2.12430555 -0.87154177]  |
|  7   | [-0.27598904 -0.05514699] | 19.15574830883937  |   

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:22: RuntimeWarning: overflow encountered in exp


+------+---------------------------+--------------------+-------------------------------+
| iter |             x0            |       f(x0)        |            fdx(x0)            |
+------+---------------------------+--------------------+-------------------------------+
|  1   | [-0.4742196   0.59764153] | 23.90281915772737  | [-921.38724761 -251.47404574] |
|  2   | [-0.37518562  0.36259496] | 22.124236704487842 |   [ 0.4951699  -1.17523284]   |
|  3   | [-0.12288636  0.10035376] | 21.112647797809693 |   [ 0.25229926 -0.2622412 ]   |
|  4   | [-0.02546271  0.10725766] | 21.033398866988907 |    [0.09742364 0.00690389]    |
|  5   | [-0.02470405  0.11128337] | 21.033095127330856 |    [0.00075866 0.00402571]    |
+------+---------------------------+--------------------+-------------------------------+
Metodo completado en 5 iteraciones


+ Instancia N: 8
+------+---------------------------+--------------------+-----------------------------+
| iter |             x0            |       f(x0)